In [1]:
import torch


In [66]:
gamma = 0.9
lamda = 0.9
rewards = torch.as_tensor([1,1,1,1,12,1,1,1,1,1,2], dtype=torch.float32)
dones = torch.as_tensor([0,0,0,0,1,0,0,0,0,0,1], dtype=torch.float32)
values = torch.ones(11)
successor_values = torch.ones(11)

In [67]:
def single_gae(rewards, values, successor_values, gamma, lamda):
    N = len(rewards)
    deltas = rewards + gamma * successor_values - values
    gamlam = gamma * lamda
    gamlam_geo_series = torch.as_tensor([gamlam**i
                                         for i in range(N)]) * (1 - gamlam)
    full_gamlam_matrix = torch.stack(
        [torch.roll(gamlam_geo_series, shifts=n) for n in range(N)])
    full_gamlam_matrix = torch.triu(full_gamlam_matrix)
    # make sure it sums to one:
    # (by making the term for the last value be 1 - sum(all other terms))
    full_gamlam_matrix[:, -1] = 1 - full_gamlam_matrix[:, :-1].sum(axis=1)
    return full_gamlam_matrix @ deltas

In [73]:
location_dones = (is_terminals.nonzero()[:, 0] + 1).tolist()
location_dones = [0] + location_dones + [-1]
episodes_gaes = []
for start, end in zip(location_dones[:-1], location_dones[1:]):
    if end != -1:
        episodes_gaes.append(
            single_gae(rewards[start:end], values[start:end],
                       successor_values[start:end], gamma, lamda))
    else:
        episodes_gaes.append(
            single_gae(rewards[start:], values[start:],
                       successor_values[start:], gamma, lamda))
return torch.cat(episodes_gaes)

In [93]:
location_dones = dones.nonzero()[:,0]
len_chunks = location_dones[1:] - location_dones[0]
len_chunks = len_chunks.tolist() + [N - sum(len_chunks).item()]

In [94]:
len_chunks

[6, 5]

In [88]:
len_chunks

tensor([6])

In [81]:
torch.split(rewards, location_dones)

RuntimeError: start (5) + length (11) exceeds dimension size (11).

In [76]:
rewards.shape

torch.Size([11])

In [80]:
rewards[0:]

tensor([ 1.,  1.,  1.,  1., 12.,  1.,  1.,  1.,  1.,  1.,  2.])

In [74]:
episodes_gaes

[tensor([ 5.6351,  6.7459,  8.1171,  9.8100, 11.9000]),
 tensor([1.2487, 1.3305, 1.4314, 1.5561, 1.7100, 1.9000])]

tensor([ 5.6351,  6.7459,  8.1171,  9.8100, 11.9000,  1.2487,  1.3305,  1.4314,
         1.5561,  1.7100,  1.9000])

In [65]:
rewards

tensor([ 1.,  1.,  1.,  1., 12.,  1.,  1.,  1.,  1.,  1.,  2.])

In [49]:
location_dones

tensor([[ 4],
        [10]])

In [36]:
gae(rewards, values, successor_values, dones, gamma, lamda)

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [42]:
N = len(rewards)
deltas = rewards + gamma * successor_values * (1-dones) - values
gamlam = gamma * lamda

In [43]:
gamlam_geo_series = torch.as_tensor([gamlam**i
                                     for i in range(N)]) * (1 - gamlam)

In [44]:
gamlam_geo_series

tensor([0.1000, 0.0900, 0.0810, 0.0729, 0.0656, 0.0590, 0.0531, 0.0478, 0.0430,
        0.0387, 0.0349])

In [27]:
deltas

tensor([0.9000, 0.9000, 0.9000, 0.9000, 1.0000, 0.9000, 0.9000, 0.9000, 0.9000,
        0.9000, 1.0000])

In [16]:
rewards

tensor([1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 2.])

In [18]:
successor_values

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])